<a href="https://colab.research.google.com/github/Yuvan-nanish/disaster-response-ai-agent-multi-agent-emergency-overpass/blob/main/disaster_response_ai_agent_multi_agent_emergency_overpass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction

Natural disasters such as floods, cyclones, earthquakes, and hurricanes create sudden and life-threatening situations where people must quickly access accurate information and identify safe evacuation routes. However, during these emergencies, information is scattered across multiple sources—news websites, government portals, social media, and local authorities—making it extremely difficult for individuals to make fast, informed decisions.

This project presents an AI-powered multi-agent disaster response system designed to monitor real-time alerts, evaluate geographical risks, locate essential resources, and provide clear survival instructions. The agent system is built using the concepts from the Kaggle x Google AI Agents Intensive, utilizing LLM-powered agents, parallel task execution, custom tools, memory, and observability.

Problem Statement

During disasters, people face three major challenges:

1. Difficulty verifying real-time information

Most individuals cannot manually track breaking news, warnings, evacuations, or updates from credible sources.

2. Limited understanding of local hazards

People often do not know:

Which areas are dangerous

Which zones are safe

How to navigate around blocked or flooded regions

3. Lack of knowledge about nearby emergency resources

Critical information like:

Nearest shelters

Hospitals

Emergency rooms

Relief centers
is rarely available in one place or updated in real time.

This delay in situational awareness directly impacts survival and safety.

**What the Agent Does**

The system uses four coordinated agents to process information and deliver a complete disaster response package:

> *Agent 1: Real-time News Monitor Agent*

Uses search tools / news APIs

Fetches latest disaster alerts

Verifies and summarizes news

Can run in long-running mode (polling loop)

> *Agent 2: Geo Navigation Agent*

Uses a geolocation/OpenAPI tool

Identifies safe zones and hazard zones

Computes recommended evacuation routes

> *Agent 3: Resources Agent*

Finds nearest hospitals, shelters, relief camps

Maintains long-term memory of:

last known safe point

user’s city

past disaster type

> *> Agent 4: Summary & Instruction Agent*

Uses an LLM to combine all data

Generates:

Current situation overview

Safe routes

Resource list

Safety + first-aid instructions

Produces a clean human-friendly final output

In [ ]:
import time
import logging
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

import requests  # for OpenAPI tools (if you use real APIs)


In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("disaster-agent")


In [ ]:
class InMemorySessionService:
    def __init__(self):
        self.sessions: Dict[str, Dict[str, Any]] = {}

    def get_session(self, user_id: str) -> Dict[str, Any]:
        if user_id not in self.sessions:
            self.sessions[user_id] = {
                "last_city": None,
                "last_disaster_type": None,
                "last_safe_point": None,
            }
        return self.sessions[user_id]

    def update_session(self, user_id: str, **kwargs):
        session = self.get_session(user_id)
        session.update(kwargs)


In [ ]:
session_service = InMemorySessionService()


In [ ]:
@dataclass
class Metrics:
    alerts_processed: int = 0
    scenarios_run: int = 0
    false_alerts_flagged: int = 0
    avg_response_time_sec: float = 0.0
    history: List[Dict[str, Any]] = field(default_factory=list)

    def log_scenario(self, scenario_name: str, response_time: float, alerts_count: int):
        self.scenarios_run += 1
        self.alerts_processed += alerts_count
        # simple running average
        if self.scenarios_run == 1:
            self.avg_response_time_sec = response_time
        else:
            self.avg_response_time_sec = (
                (self.avg_response_time_sec * (self.scenarios_run - 1)) + response_time
            ) / self.scenarios_run
        self.history.append({
            "scenario": scenario_name,
            "response_time": response_time,
            "alerts_count": alerts_count
        })

metrics = Metrics()


In [ ]:
!pip install newsapi-python


In [ ]:
from newsapi import NewsApiClient

# Add your API key here
NEWS_API_KEY = "38249c2acf264751a89d2cea1ffc9219"   # <-- replace with your real key
newsapi = NewsApiClient(api_key=NEWS_API_KEY)

def news_search_tool(location: str, disaster_type: str) -> List[Dict[str, Any]]:
    """
    Real NewsAPI integration.
    Fetches the latest news articles related to the disaster and location.
    """
    query = f"{location} {disaster_type}"

    try:
        articles = newsapi.get_everything(
            q=query,
            language='en',
            sort_by='publishedAt',
            page_size=5
        )

        results = []
        for a in articles.get("articles", []):
            results.append({
                "title": a["title"],
                "source": a["source"]["name"],
                "summary": a["description"],
                "url": a["url"]
            })

        if not results:
            print("[news_search_tool] No real news articles found.")
        else:
            print(f"[news_search_tool] Found {len(results)} real news articles.")

        return results

    except Exception as e:
        print(f"[news_search_tool] Error fetching real news: {e}")
        return []


In [ ]:
def geo_safe_route_tool(location: str, disaster_type: str) -> Dict[str, Any]:
    """
    Simulated OpenAPI tool for maps/geo.
    Returns safe and hazard zones along with a suggested route.
    """
    logger.info(f"[Tool] Computing safe route for {disaster_type} in {location}")
    return {
        "safe_areas": [
            {"name": "City Hall Relief Center", "description": "Higher ground, official camp"},
            {"name": "Central School Shelter", "description": "Designated shelter point"}
        ],
        "hazard_zones": [
            {"name": "Riverfront Area", "description": "High flood risk"},
        ],
        "recommended_route": "Avoid Riverfront Area, move via Main Road to City Hall Relief Center."
    }


In [ ]:
def nearby_resources_tool(location: str) -> Dict[str, List[Dict[str, Any]]]:
    """
    Simulated OpenAPI for nearby hospitals and shelters.
    """
    logger.info(f"[Tool] Fetching resources near {location}")
    return {
        "hospitals": [
            {"name": "General Hospital", "distance_km": 2.1, "emergency_available": True},
            {"name": "City Medical Center", "distance_km": 3.5, "emergency_available": False},
        ],
        "shelters": [
            {"name": "Community Hall Shelter", "distance_km": 1.2, "capacity": "High"},
            {"name": "School Auditorium Shelter", "distance_km": 2.8, "capacity": "Medium"},
        ],
    }


In [ ]:
class NewsMonitorAgent:
    def __init__(self):
        pass

    def run(self, location: str, disaster_type: str) -> List[Dict[str, Any]]:
        news = news_search_tool(location, disaster_type)
        logger.info(f"[NewsMonitorAgent] Found {len(news)} news items")
        return news


In [ ]:
    def run_polling(self, location: str, disaster_type: str, iterations: int = 3, delay_sec: int = 5):
        """
        Simulates a long-running monitor with pause/resume.
        """
        all_alerts = []
        for i in range(iterations):
            logger.info(f"[NewsMonitorAgent] Poll iteration {i+1}")
            alerts = self.run(location, disaster_type)
            all_alerts.extend(alerts)
            time.sleep(delay_sec)
        return all_alerts


In [ ]:
class GeoNavigationAgent:
    def __init__(self):
        pass

    def run(self, location: str, disaster_type: str) -> Dict[str, Any]:
        geo_info = geo_safe_route_tool(location, disaster_type)
        logger.info(f"[GeoNavigationAgent] Safe areas: {len(geo_info['safe_areas'])}, "
                    f"Hazard zones: {len(geo_info['hazard_zones'])}")
        return geo_info


In [ ]:
class ResourcesAgent:
    def __init__(self, session_service: InMemorySessionService):
        self.session_service = session_service

    def run(self, user_id: str, location: str) -> Dict[str, Any]:
        session = self.session_service.get_session(user_id)
        session["last_city"] = location

        resources = nearby_resources_tool(location)
        # Update "last safe point" as first shelter for example
        if resources["shelters"]:
            session["last_safe_point"] = resources["shelters"][0]["name"]

        logger.info(f"[ResourcesAgent] Found {len(resources['hospitals'])} hospitals and "
                    f"{len(resources['shelters'])} shelters")
        return resources


In [ ]:
def fake_llm_generate(prompt: str) -> str:
    # Placeholder. Replace with a real LLM call in your environment.
    return "SIMULATED LLM RESPONSE:\n" + prompt[:1000]


In [ ]:
class SummaryInstructionAgent:
    def __init__(self):
        pass

    def run(self, location: str, disaster_type: str,
            news: List[Dict[str, Any]],
            geo_info: Dict[str, Any],
            resources: Dict[str, Any]) -> str:
        prompt = f"""
You are a disaster response assistant.

User location: {location}
Disaster type: {disaster_type}

Latest alerts:
{news}

Geo information (safe and hazard zones, routes):
{geo_info}

Nearby resources (hospitals, shelters):
{resources}

Task:
1. Briefly summarize the current situation in {location}.
2. Provide a recommended safe route and safe areas.
3. List nearest hospitals and shelters with very short descriptions.
4. Give 5-8 bullet points of safety instructions and basic first-aid tips for this type of disaster.
5. Use clear, calm, human-friendly language.
"""
        response = fake_llm_generate(prompt)
        logger.info("[SummaryInstructionAgent] Generated response for user")
        return response


In [ ]:
class DisasterResponseOrchestrator:
    def __init__(self, session_service: InMemorySessionService, metrics: Metrics):
        self.news_agent = NewsMonitorAgent()
        self.geo_agent = GeoNavigationAgent()
        self.resources_agent = ResourcesAgent(session_service)
        self.summary_agent = SummaryInstructionAgent()
        self.metrics = metrics

    def handle_request(self, user_id: str, location: str, disaster_type: str,
                       scenario_name: str = "default_scenario") -> str:
        start = time.time()

        # In real code, you could run news_agent and geo_agent in parallel threads
        news = self.news_agent.run(location, disaster_type)
        geo_info = self.geo_agent.run(location, disaster_type)
        resources = self.resources_agent.run(user_id, location)

        response = self.summary_agent.run(location, disaster_type, news, geo_info, resources)

        end = time.time()
        self.metrics.log_scenario(
            scenario_name=scenario_name,
            response_time=end - start,
            alerts_count=len(news)
        )
        return response


In [ ]:
orchestrator = DisasterResponseOrchestrator(session_service, metrics)

user_id = "demo_user_1"
location = "Chennai, India"
disaster_type = "flood"

final_response = orchestrator.handle_request(
    user_id=user_id,
    location=location,
    disaster_type=disaster_type,
    scenario_name="Chennai_flood_demo"
)

print(final_response)


[news_search_tool] Found 5 real news articles.
SIMULATED LLM RESPONSE:

You are a disaster response assistant.

User location: Chennai, India
Disaster type: flood

Latest alerts:
[{'title': 'DMK allies to protest against SIR on Nov 11', 'source': 'The Times of India', 'summary': 'CHENNAI: The DMK-led secular progressive alliance will protest against Election Commission of India over the ongoing special intensive revision (SIR) .', 'url': 'https://timesofindia.indiatimes.com/city/chennai/dmk-allies-to-protest-against-sir-on-nov-11/articleshow/125140194.cms'}, {'title': 'Sponge parks at Rupnagar, Kahilipara to help mitigate flooding in Guwahati', 'source': 'The Times of India', 'summary': 'Guwahati is pioneering sponge parks, a nature-based flood mitigation strategy, at Rupnagar and Kahilipara. These engineered wetlands, designed by AUKH and implemented by GMDA, will absorb and release rainwater, improving groundwater and urban ecosystems. This…', 'url': 'https://timesofindia.indiatimes.

## 8. Real API Integration (Overpass + Nominatim)

In this section, we upgrade the agent from simulated tools to **real-world geographic data** using:

- **Nominatim (OpenStreetMap)** for geocoding (location name → latitude/longitude).
- **Overpass API** for querying real OpenStreetMap features such as hospitals, shelters, and water/hazard zones.

> Note: No API key is required for either Nominatim or Overpass. These are free, public services, but they must be used responsibly (with rate limiting and proper User-Agent headers).

In [ ]:
import requests
from typing import List, Dict, Any, Optional, Tuple

# Reusable headers for Nominatim (required by usage policy)
OSM_HEADERS = {
    "User-Agent": "DisasterResponseAgent/1.0 (Kaggle Notebook; contact: example@example.com)"
}

In [ ]:
OPEN_CAGE_KEY = "cd0dc66e37b4407e9eab500d2dfa49d0"

def geocode_location_osm(location):
    url = "https://api.opencagedata.com/geocode/v1/json"
    params = {"q": location, "key": OPEN_CAGE_KEY}

    r = requests.get(url, params=params)
    data = r.json()

    lat = data["results"][0]["geometry"]["lat"]
    lon = data["results"][0]["geometry"]["lng"]
    return lat, lon


In [ ]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

OVERPASS_SERVERS = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass.nchc.org.tw/api/interpreter"
]

def _run_overpass_query(query: str):
    """
    Robust Overpass client with automatic retries across multiple servers.
    Prevents 429 and 504 errors from breaking the pipeline.
    """
    for server in OVERPASS_SERVERS:
        try:
            print(f"[Overpass] Trying server: {server}")
            r = requests.post(server, data={'data': query}, timeout=40)
            r.raise_for_status()
            print(f"[Overpass] Success on {server}")
            return r.json()

        except Exception as e:
            print(f"[Overpass] Server failed ({server}): {e}")
            continue

    print("[Overpass] All servers failed.")
    return None


def get_hospitals_overpass(lat: float, lon: float, radius_m: int = 5000) -> List[Dict[str, Any]]:
    """Return nearby hospitals as a list of OSM elements."""
    query = f"""
    [out:json];
    (
      node["amenity"="hospital"](around:{radius_m},{lat},{lon});
      way["amenity"="hospital"](around:{radius_m},{lat},{lon});
      relation["amenity"="hospital"](around:{radius_m},{lat},{lon});
    );
    out center;
    """
    data = _run_overpass_query(query)
    return data.get("elements", []) if data else []


def get_shelters_overpass(lat: float, lon: float, radius_m: int = 5000) -> List[Dict[str, Any]]:
    """Return nearby shelters as a list of OSM elements."""
    query = f"""
    [out:json];
    (
      node["amenity"="shelter"](around:{radius_m},{lat},{lon});
      way["amenity"="shelter"](around:{radius_m},{lat},{lon});
      relation["amenity"="shelter"](around:{radius_m},{lat},{lon});
    );
    out center;
    """
    data = _run_overpass_query(query)
    return data.get("elements", []) if data else []


def get_flood_related_areas(lat: float, lon: float, radius_m: int = 5000) -> List[Dict[str, Any]]:
    """Return OSM features potentially related to flood/hazard/water in the area."""
    query = f"""
    [out:json];
    (
      way["hazard"="flood"](around:{radius_m},{lat},{lon});
      way["flood_prone"="yes"](around:{radius_m},{lat},{lon});
      way["natural"="water"](around:{radius_m},{lat},{lon});
    );
    out center;
    """
    data = _run_overpass_query(query)
    return data.get("elements", []) if data else []

### 9. Updating Agents to Use Real Overpass/Nominatim Data

Next, we redefine the `GeoNavigationAgent` and `ResourcesAgent` so that they:

- Use **Nominatim** to convert the user's location (e.g. `"Chennai, India"`) into latitude/longitude.
- Use **Overpass API** to fetch:
  - Nearby hospitals
  - Nearby shelters
  - Flood-related or water areas (for hazard context)

These new definitions override the earlier simulated versions when the notebook is run from top to bottom.

In [ ]:
class ResourcesAgent:
    """Agent responsible for discovering nearby hospitals and shelters using Overpass API.

    It also updates session memory with:
    - last_city
    - last_safe_point (first shelter found, if any)
    """
    def __init__(self, session_service):
        self.session_service = session_service

    def run(self, user_id: str, location: str) -> Dict[str, Any]:
        session = self.session_service.get_session(user_id)
        session["last_city"] = location

        geo = geocode_location_osm(location)
        if geo is None:
            print(f"[ResourcesAgent] Could not geocode location: {location}")
            hospitals = []
            shelters = []
        else:
            lat, lon = geo
            hospitals = get_hospitals_overpass(lat, lon)
            shelters = get_shelters_overpass(lat, lon)

        def _summarize_resource(elem):
            tags = elem.get("tags", {})
            name = tags.get("name", "Unnamed")
            amenity = tags.get("amenity", "unknown")
            return {
                "name": name,
                "amenity": amenity,
                "lat": elem.get("lat") or elem.get("center", {}).get("lat"),
                "lon": elem.get("lon") or elem.get("center", {}).get("lon"),
            }

        hospitals_summary = [_summarize_resource(e) for e in hospitals]
        shelters_summary = [_summarize_resource(e) for e in shelters]

        if shelters_summary:
            session["last_safe_point"] = shelters_summary[0]["name"]

        resources = {
            "hospitals": hospitals_summary,
            "shelters": shelters_summary,
        }

        print(f"[ResourcesAgent] Found {len(hospitals_summary)} hospitals and {len(shelters_summary)} shelters")
        return resources

In [ ]:
class GeoNavigationAgent:
    """Agent that provides basic geo-context for the disaster using Overpass data."""
    def __init__(self):
        pass

    def run(self, location: str, disaster_type: str) -> Dict[str, Any]:
        geo = geocode_location_osm(location)
        if geo is None:
            print(f"[GeoNavigationAgent] Could not geocode location: {location}")
            return {
                "safe_areas": [],
                "hazard_zones": [],
                "recommended_route": "Unable to compute route due to missing geolocation.",
            }

        lat, lon = geo
        flood_elems = get_flood_related_areas(lat, lon)

        hazard_zones = []
        for e in flood_elems:
            tags = e.get("tags", {})
            name = tags.get("name", "Water / flood-prone area")
            hazard_zones.append({
                "name": name,
                "lat": e.get("lat") or e.get("center", {}).get("lat"),
                "lon": e.get("lon") or e.get("center", {}).get("lon"),
                "description": "Potential flood or water-related hazard zone from OSM data."
            })

        safe_areas = [
            {
                "name": "Higher ground / central civic buildings",
                "description": "Move towards higher elevation and official relief centers where available."
            }
        ]

        recommended_route = (
            "Avoid low-lying areas and regions close to mapped water bodies. "
            "Move towards higher ground and official shelters if indicated in the resources."
        )

        print(f"[GeoNavigationAgent] Hazard zones identified: {len(hazard_zones)}")
        return {
            "safe_areas": safe_areas,
            "hazard_zones": hazard_zones,
            "recommended_route": recommended_route,
        }

### 10. News Monitor Agent – Placeholder for Real News API

Below we keep a **placeholder** `news_search_tool` so that you can plug in any real news API of your choice
(e.g. NewsAPI, GNews, etc.).

For now, it returns an empty list by default, and the `NewsMonitorAgent` will handle that gracefully.

In [ ]:
def news_search_tool(location: str, disaster_type: str):
    """Placeholder for a real news API integration.

    You can replace this implementation with:
    - NewsAPI.org
    - GNews
    - any other trusted news source.

    Expected return format:
        List[{
            "title": str,
            "source": str,
            "summary": str,
            "url": str
        }]
    """
    print("[news_search_tool] No real news API configured yet. Returning empty list.")
    return []


class NewsMonitorAgent:
    def __init__(self):
        pass

    def run(self, location: str, disaster_type: str):
        news = news_search_tool(location, disaster_type)
        print(f"[NewsMonitorAgent] Retrieved {len(news)} news items")
        return news

### 11. Improved Summary & Instruction Agent

We now slightly improve the `SummaryInstructionAgent` prompt so that it:

- Explicitly refers to hospitals, shelters, and hazard zones.
- Produces structured, user-friendly output.
- Handles the case where some information may be missing (e.g., no shelters found).

In [ ]:
def fake_llm_generate(prompt: str) -> str:
    """Placeholder LLM call.

    In a real deployment, this would call an LLM.
    For this notebook, we simply echo the prompt prefix so the pipeline is visible.
    """
    return "LLM RESPONSE (placeholder)\n\n" + prompt[:1200]


class SummaryInstructionAgent:
    def __init__(self):
        pass

    def run(
        self,
        location: str,
        disaster_type: str,
        news: List[Dict[str, Any]],
        geo_info: Dict[str, Any],
        resources: Dict[str, Any],
    ) -> str:
        prompt = f"""
You are a calm, concise disaster response assistant.

User location: {location}
Disaster type: {disaster_type}

Latest alerts (may be empty):
{news}

Geo information (safe areas, hazard zones, general route guidance):
{geo_info}

Nearby resources (hospitals and shelters from OpenStreetMap):
{resources}

Your task:
1. Give a 2–3 sentence situation summary for {location}.
2. List any known hazard zones or flood-prone areas. If none are available, say that explicit hazard zones are not mapped.
3. List the nearest hospitals and shelters in short bullet points (name + one helpful detail).
4. Provide 5–8 clear safety instructions tailored to a {disaster_type}.
5. If information is missing (e.g., no shelters found), tell the user what they should do instead (e.g., contact authorities, move to higher ground).
6. Use simple, reassuring language. Do NOT mention that this came from a model or an API; just speak as an assistant.
"""
        response = fake_llm_generate(prompt)
        print("[SummaryInstructionAgent] Generated guidance text.")
        return response

### 12. Updated Orchestrator and Evaluation with Real Data

Finally, we redefine the `DisasterResponseOrchestrator` so that it uses:

- The new `NewsMonitorAgent` (with a pluggable real news API).
- The new `GeoNavigationAgent` (using Nominatim + Overpass).
- The new `ResourcesAgent` (using Overpass).
- The improved `SummaryInstructionAgent`.

We also run a few evaluation scenarios and log metrics like response time and number of alerts processed.

In [ ]:
import time
from dataclasses import dataclass, field

@dataclass
class Metrics:
    alerts_processed: int = 0
    scenarios_run: int = 0
    avg_response_time_sec: float = 0.0
    history: List[Dict[str, Any]] = field(default_factory=list)

    def log_scenario(self, scenario_name: str, response_time: float, alerts_count: int):
        self.scenarios_run += 1
        self.alerts_processed += alerts_count
        if self.scenarios_run == 1:
            self.avg_response_time_sec = response_time
        else:
            self.avg_response_time_sec = (
                (self.avg_response_time_sec * (self.scenarios_run - 1)) + response_time
            ) / self.scenarios_run
        self.history.append(
            {
                "scenario": scenario_name,
                "response_time_sec": response_time,
                "alerts_count": alerts_count,
            }
        )


class InMemorySessionService:
    def __init__(self):
        self.sessions: Dict[str, Dict[str, Any]] = {}

    def get_session(self, user_id: str) -> Dict[str, Any]:
        if user_id not in self.sessions:
            self.sessions[user_id] = {
                "last_city": None,
                "last_disaster_type": None,
                "last_safe_point": None,
            }
        return self.sessions[user_id]

    def update_session(self, user_id: str, **kwargs):
        session = self.get_session(user_id)
        session.update(kwargs)


metrics = Metrics()
session_service = InMemorySessionService()


class DisasterResponseOrchestrator:
    def __init__(self, session_service: InMemorySessionService, metrics: Metrics):
        self.news_agent = NewsMonitorAgent()
        self.geo_agent = GeoNavigationAgent()
        self.resources_agent = ResourcesAgent(session_service)
        self.summary_agent = SummaryInstructionAgent()
        self.metrics = metrics

    def handle_request(
        self,
        user_id: str,
        location: str,
        disaster_type: str,
        scenario_name: str = "default_scenario",
    ) -> str:
        start = time.time()

        news = self.news_agent.run(location, disaster_type)
        geo_info = self.geo_agent.run(location, disaster_type)
        resources = self.resources_agent.run(user_id, location)
        response = self.summary_agent.run(location, disaster_type, news, geo_info, resources)

        end = time.time()
        self.metrics.log_scenario(
            scenario_name=scenario_name,
            response_time=end - start,
            alerts_count=len(news),
        )
        return response

In [ ]:
# Instantiate orchestrator with real-data-capable agents
orchestrator = DisasterResponseOrchestrator(session_service, metrics)

demo_scenarios = [
    ("user1", "Chennai, India", "flood", "Chennai_flood_demo"),
    ("user2", "Tokyo, Japan", "earthquake", "Tokyo_earthquake_demo"),
    ("user3", "Miami, USA", "hurricane", "Miami_hurricane_demo"),
]

for user_id, loc, dtype, name in demo_scenarios:
    print("\n" + "=" * 80)
    print(f"Scenario: {name} | Location: {loc} | Disaster: {dtype}")
    guidance = orchestrator.handle_request(user_id, loc, dtype, scenario_name=name)
    print(guidance[:800] + "\n...\n")


Scenario: Chennai_flood_demo | Location: Chennai, India | Disaster: flood
[news_search_tool] No real news API configured yet. Returning empty list.
[NewsMonitorAgent] Retrieved 0 news items
[Overpass] Trying server: https://overpass-api.de/api/interpreter
[Overpass] Server failed (https://overpass-api.de/api/interpreter): 504 Server Error: Gateway Timeout for url: https://overpass-api.de/api/interpreter
[Overpass] Trying server: https://lz4.overpass-api.de/api/interpreter
[Overpass] Success on https://lz4.overpass-api.de/api/interpreter
[GeoNavigationAgent] Hazard zones identified: 80
[Overpass] Trying server: https://overpass-api.de/api/interpreter
[Overpass] Server failed (https://overpass-api.de/api/interpreter): 504 Server Error: Gateway Timeout for url: https://overpass-api.de/api/interpreter
[Overpass] Trying server: https://lz4.overpass-api.de/api/interpreter
[Overpass] Success on https://lz4.overpass-api.de/api/interpreter
[Overpass] Trying server: https://overpass-api.de/api/

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame(metrics.history)
metrics_df

,scenario,response_time_sec,alerts_count
0,Chennai_flood_demo,23.152673,0
1,Tokyo_earthquake_demo,11.898229,0
2,Miami_hurricane_demo,11.178340,0


In [ ]:
resources = orchestrator.resources_agent.run("user1", "Chennai, India")


[Overpass] Trying server: https://overpass-api.de/api/interpreter
[Overpass] Server failed (https://overpass-api.de/api/interpreter): 504 Server Error: Gateway Timeout for url: https://overpass-api.de/api/interpreter
[Overpass] Trying server: https://lz4.overpass-api.de/api/interpreter
[Overpass] Success on https://lz4.overpass-api.de/api/interpreter
[Overpass] Trying server: https://overpass-api.de/api/interpreter
[Overpass] Success on https://overpass-api.de/api/interpreter
[ResourcesAgent] Found 215 hospitals and 55 shelters


In [ ]:
def print_resources(resources):
    print("\n=== Nearby Hospitals ===")
    for h in resources["hospitals"]:
        print(f"- {h['name']}  (lat={h['lat']}, lon={h['lon']})")

    print("\n=== Nearby Shelters ===")
    for s in resources["shelters"]:
        print(f"- {s['name']}  (lat={s['lat']}, lon={s['lon']})")




In [ ]:

print_resources(resources)


=== Nearby Hospitals ===
- Amrit Hospital  (lat=13.0922007, lon=80.2792912)
- Vikram Hospital  (lat=13.0985007, lon=80.2796178)
- Goverment dental hospital & collage  (lat=13.0829605, lon=80.281264)
- Kilpauk Medical College  (lat=13.0785984, lon=80.2433369)
- Govt Maternity Hospital  (lat=13.0720996, lon=80.2589871)
- Institute of Child Health  (lat=13.0735496, lon=80.257025)
- Institute of Obstetrics and Gynaecology dd  (lat=13.0735431, lon=80.2591463)
- Rajan Eye Care Hospital  (lat=13.0508335, lon=80.2434416)
- Amma Hospital  (lat=13.0626045, lon=80.2305209)
- Right Hospitals  (lat=13.0787952, lon=80.2405731)
- Dr Venugopals' Sundaram Clini  (lat=13.076782, lon=80.268345)
- Dr Kabir's Clinic  (lat=13.0768734, lon=80.2678381)
- Dr Sridhars'Clinic  (lat=13.0764345, lon=80.2672427)
- Vepery Hospital  (lat=13.0921812, lon=80.2594411)
- Shankara Netralaya Eye Hospital  (lat=13.0642564, lon=80.2497746)
- Unnamed  (lat=13.1133021, lon=80.2905249)
- Unnamed  (lat=13.1131363, lon=80.242936